In [1]:
import requests
import json

In [2]:
def extract():
    f = open('tmdb.json')
    if f:
        return json.loads(f.read())

In [3]:
print("Extracting data");
data = extract();
print("Extraction complete");

Extracting data
Extraction complete


In [9]:
def reindex(analysisSettings={}, mappingSettings={}, movieDict={}):
    settings = {
        "settings": {
            "number_of_shards": 1,
            "index": {
                "analysis": analysisSettings,
            }
        }
    }
    headers = {
        "Content-Type": "application/json"
    }
    if mappingSettings:
        settings["mappings"] = mappingSettings
    
    resp = requests.delete("http://localhost:9200/tmdb")
    print("Delete: ", resp)
    print("settings: ", json.dumps(settings))
    resp = requests.put("http://localhost:9200/tmdb",headers=headers, data=json.dumps(settings))
    print("Put settings: ", resp.status_code, resp.json())
    
    bulkMovies = ""
    
    for id, movie in movieDict.items():
        addCmd = {"index": {"_index": "tmdb",
                           "_type": "movie",
                           "_id": movie["id"]}}
        bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(movie) + "\n"
    resp = requests.post("http://localhost:9200/_bulk",headers=headers, data=bulkMovies)
    print("Indexing complete")

In [7]:
reindex(movieDict=data)

Delete:  <Response [200]>
settings:  {"settings": {"number_of_shards": 1, "index": {"analysis": {}}}}
Put settings:  200 {'acknowledged': True, 'shards_acknowledged': True, 'index': 'tmdb'}
200 {'took': 5810, 'errors': False, 'items': [{'index': {'_index': 'tmdb', '_type': 'movie', '_id': '93837', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'tmdb', '_type': 'movie', '_id': '8193', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'tmdb', '_type': 'movie', '_id': '8195', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'tmdb', '_type': 'movie', '_id': '5', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '

In [10]:
def search(query):
    headers = {
        "Content-Type": "application/json"
    }
    url = "http://localhost:9200/tmdb/movie/_search"
    httpResp = requests.get(url, headers=headers, data=json.dumps(query))
    searchHits = json.loads(httpResp.text)['hits']
    print("Num\tRelevance Score\t\tMovie Title")
    for idx, hit in enumerate(searchHits["hits"]):
        print("%s\t%s\t\t%s" %
        (idx + 1, hit['_score'], hit['_source']["title"]))

In [11]:
userSearch = 'basketball with cartoon aliens'
query = {
    "query": {
        "multi_match": {
            "query": userSearch,
            "fields": ["title^10", "overview"]
        }
    }
}

In [12]:
search(query)

Num	Relevance Score		Movie Title
1	85.56929		Aliens
2	73.71077		The Basketball Diaries
3	71.3202		Cowboys & Aliens
4	61.13922		Monsters vs Aliens
5	53.501823		Aliens vs Predator: Requiem
6	53.501823		Aliens in the Attic
7	45.221096		Dances with Wolves
8	45.221096		Friends with Benefits
9	45.221096		Fire with Fire
10	45.221096		Friends with Kids
